## Labwork 5

## Detection using matched filter

## Contents

1 Characterizing the detector performance . . . . . . . . 20
2 Object detection in a real-time video . . . . . . . . . . . 22
2.1 Matched Filter on images . . . . . . . . . . . . . . . . . 22
2.2 Real-time Video . . . . . . . . . . . . . . . . . . . . . . . 23

In radar detection, one must test two hypotheses :
Hypothesis H0 $\quad x(t)=b(t)$
Hypothesis H1 $\quad x(t)=a s(t)+b(t)$
where :

- $x(t)$ is the measured signal,
- $s(t)$ is the emitted pulse, known by the user,
- $a$ is the attenuation factor,
- $b(t)$ is the noise on the measure.



We assume that we know where the target may be. We only want to know if the target is or not at this position.

We first only consider the detection problem. In reality, we don't know if there is a target, neither its possible position. We must detect and estimate at the same time. That is what we will study in the image processing application.

If the noise $b(t)$ is white, the optimal detector (matched filter) is a correlation between the measured signal and the emitted pulse :

$$
F(\theta)=\int x(t) s(t-\theta) d t
$$

We only need to calculate this expression at one point (chosen here as 0 ) if we assume that the position is known. Thus, we have to test :

$$
F(0)=\int x(t) s(t) d t
$$

This value is then compared to a threshold in order to take a decision.

![](https://cdn.mathpix.com/cropped/2024_09_27_5764cda6987cdb496128g-26.jpg?height=235&width=1235&top_left_y=1002&top_left_x=365)



## 1 Characterizing the detector performance

The SNR (Signal to Noise Ratio) of the detection is defined by :

$$
\mathrm{SNR}=\frac{a^{2} E_{s}}{\sigma^{2}} \quad \mathrm{SNR}_{\mathrm{dB}}=10 \log \left(\frac{a^{2} E_{s}}{\sigma^{2}}\right)
$$

where $E_{s}$ is the energy of the emitted pulse: $E_{s}=\int|s(t)|^{2} d t$ and $\sigma^{2}$ the variance of the noise $b(t)$.

We assume that the temporal shape of the pulse is gaussian and the attenuation factor $a=1$. We suppose also that $b(t)$ is a gaussian white noise.
$\rightsquigarrow$ Design a Matlab function which gives a realization of the noisy signal $x$. Length of the vector will be chosen as 255 , duration of the pulse as 40 samples and SNR as 12 dB .
$\rightsquigarrow$ Plot the noisy pulse and the noise alone (hypothesis 0 and 1 ). Apply the matched filter to these two signals.


In [512]:
# Import libraries
import numpy as np
import scipy as sp
import plotly.graph_objects as go

In [513]:
# Define generative functions 

def gaussian_pulse(X, noise_std):
    """
    returns a 2D array of clean signal as a function of time and different decay rate.
    """

    #e = scipy.gausspulse(X, fc=5, retquad=False, retenv=True)
    e = sp.signal.windows.gaussian(len(X), noise_std)

    return e



def noisy_pulse(X, SNR, mean, noise_std, N):
    """
    returns a 2D array of clean signal as a function of time and different decay rate, with the SAME white gaussian noise.
    """

    # Générer le signal exponentiel
    clean_signal = gaussian_pulse(X, noise_std)

    # Compute sigma_s
    E = np.sum(np.power(clean_signal,2))
    noise_std = np.sqrt(E/SNR)

    # Ajouter du bruit gaussien
    noise_1D = np.random.normal(mean, noise_std, size=N)
    #noise = np.vstack([noise_1D] * len(noise_1D))

    #res = clean_signal + noise

    return noise_1D, np.add(clean_signal, noise_1D) #res

In [514]:
# Define plot function

def plot(X, function, plotName, show):
    # Create the plot
    fig = go.Figure()

    # Add error bars
    fig.add_trace(go.Scatter(
        x=X,
        y=function,
        mode='markers+lines',
        name='Noisy signal',
        #error_y=dict(type='data', array=std_devs, visible=True),
        marker=dict(size=4)
    ))

    # Customize layout
    fig.update_layout(
        title=plotName,
        xaxis_title='time',
        yaxis_title='Signal',
        #yaxis=dict(title='LLHs', range=[0, max(LLHs) + 1]),
        #xaxis=dict(title='Decay Rate (alpha)', tickvals=decay_rate),
        showlegend=True
    )

    # Show the plot
    if show:
        fig.show()
    else:
        pass
    
    return fig

In [515]:
# Define run test function

def run():

    # Define gaussian pulse (determinist function)
    pulse_duration = 40
    sigma_s = pulse_duration/2
    mean_s = 0 # The pulse is not located in a specific place, we will be moving around a window to detect it

    # Define gaussian white noise (probabilist distribution)
    #sigma_b = 1 # The standard deviation is computed thanks to the energy of the gaussian pulse to match with the SNR constrain
    mean_b = 0

    # Define amplitude
    a = 1 # x = a * s + b
    SNR = 12 # dB

    # Defines the time vector
    N = 255 # Length of RV X (Number of samples of the signals)
    X = np.linspace(0, N, N)

    # Computes the signals
    s = gaussian_pulse(X, sigma_s)
    noise, x = noisy_pulse(X, SNR, mean_b, sigma_s, N)

    # Print the gaussian pulse
    pulseFig = plot(X, s, "Clean signal (gaussian pulse)", True)
    

    # Print the noise alone and then the noisy signal
    noiseFig = plot( X, noise, "Noise alone", True)
    noisyFig = plot(X, x, "Noisy signal, gaussian pulse perturbed (SNR = 12) by a white gaussian distribution.", True)

    # Apply the match filter
    detection = sp.signal.correlate(s, x, "same", "fft")

    # Print the resulting detection
    detectFig = plot(X, detection, "Signal resulting of the optimal filtering performed by the matched filter", True)

# Execute run test function

if __name__=="__main__":
    run()


In [516]:
# Define plot add trace function

def add_fig(fig, abs, func, newTraceTitle, title, show):
    
    fig.add_trace(go.Scatter(
        x=abs,
        y=func,
        mode='markers+lines',
        name=newTraceTitle,
        #error_y=dict(type='data', array=std_devs, visible=True),
        marker=dict(size=4)
    ))

    fig.update_layout(
        title=title,
        xaxis_title='time',
        yaxis_title='signal',
        #yaxis=dict(title='LLHs', range=[0, max(LLHs) + 1]),
        #xaxis=dict(title='Decay Rate (alpha)', tickvals=decay_rate),
        showlegend=True
    )
    if show:
        fig.show()
    else:
        pass
    
    return fig

In [517]:
# Define run test function

def run():

    # Define gaussian pulse (determinist function)
    pulse_duration = 40
    sigma_s = pulse_duration/2
    mean_s = 0 # The pulse is not located in a specific place, we will be moving around a window to detect it

    # Define gaussian white noise (probabilist distribution)
    #sigma_b = 1 # The standard deviation is computed thanks to the energy of the gaussian pulse to match with the SNR constrain
    mean_b = 0

    # Define amplitude
    a = 1 # x = a * s + b
    SNR = 12 # dB

    # Defines the time vector
    N = 255 # Length of RV X (Number of samples of the signals)
    X = np.linspace(0, N, N)

    # Computes the signals
    s = gaussian_pulse(X, sigma_s)
    noise, x = noisy_pulse(X, SNR, mean_b, sigma_s, N)

    # Apply the match filter
    detection = sp.signal.correlate(s, x, "same", "fft")
    fake_alarm = sp.signal.correlate(s, noise, "same", "fft")

    # Print the resulting detection
    detectFig = plot(X, detection, "Signal resulting of the optimal filtering performed by the matched filter", False)
    comparisonFig = add_fig(detectFig, X, fake_alarm, "Fake Alarm","True detection and Fake Alarm resulting of the mathed filter", False)

    # Computes the difference between the two situations
    meanDetect = np.mean(detection)
    meanD = np.ones(N) * meanDetect

    meanFake = np.mean(fake_alarm)
    meanF = np.ones(N) * meanFake

    diff = np.ones(N) * (meanDetect - meanFake)
    print(f"Mean fake alarm : {meanFake}, Mean detection : {meanDetect}, Difference : {meanFake - meanDetect}")

    tempA = add_fig(comparisonFig, X, meanF, "Mean Fake Alarm", "N/A", False)
    tempB = add_fig(comparisonFig, X, meanD, "Mean Detection", "N/A", False)
    comparison2Fig = add_fig(comparisonFig, X, diff, "Difference between detection and Fake alarm means","True detection and Fake Alarm resulting of the mathed filter", True)
    
    # Apply the match filter (no translation)
    detection_0 = sp.signal.correlate(s, x, "valid", "fft")
    fake_alarm_0 = sp.signal.correlate(s, noise, "valid", "fft")

    # Print resulting values
    print(f"Correlation in the case of a fake alarm : {fake_alarm_0} \n Correlation in the case of a detection : {detection_0}.")
    
# Execute run test function

if __name__=="__main__":
    run()


Mean fake alarm : 7.413778949546468, Mean detection : 17.269691413639187, Difference : -9.855912464092718


Correlation in the case of a fake alarm : [6.90290417] 
 Correlation in the case of a detection : [42.35198119].


<font color="yellow">
We run the script several times, and we observe that the mean of the Fake Alarm signal seem always inferior to the difference between the means of the FA signal and the Detection signal (which remain constant over iterations, approximately equals to 10). On the contrary, the mean of the Detection signal is sometimes superior, sometimes inferior to this threshold.

As a result, one can define the difference between the means as a threshold to compare the Fake Alarm mean to.

Over the iteration, one can also observe that the mean of the Detection signal seems always superior to the mean of the Fake Alarm signal. Thus, one could try to see if those means converge toward a certain value over the iterations in order to see if another threshold could be defined.

However, one could notice that here, the noisy signal is made of the clean signal without any translation over time. Thus, it is not needed to compute the whole correlation of the signal : one could simply look at the value of the correlation function at $\theta = 0 $ (i.e. no translation at all). In this case, it seems that 25 is a suitable threshold.

Notice that in the case of a translated clean signal, one could look at the maximum of the correlation function to estimate the value of the translation in the case of a detection.
</font>


Q1 Compare the two results. What threshold value do you choose?
$\rightsquigarrow$ Run your code several times.

Q2 What do you think of the threshold value first chosen ?

## Useful commands/tips:

- gausswin( $n, n /$ duration) gaussian pulse in a vector of length $n$. The duration is defined at $1 / \sqrt{\mathrm{e}} \approx 0.6$.
- $\operatorname{sum}(\mathrm{s} . \wedge 2$ ) to compute the energy of a vector s
- Calculating the cross-correlation at 0 can be easily done by a scalar product of two line vectors as $\mathrm{c}=1 / \mathrm{N} * \mathrm{x} * \mathrm{~s}^{\prime}$;
$\rightsquigarrow$ In order to carry out a Monte Carlo simulation, design two matrices (under H0 and H1) of 1000 realizations of the signal.


## Useful commands/tips:

- ones $(1000,1) * \operatorname{s}+r a n d n(1000, n)$ where $s$ is a line vector of dimension n corresponding to the emitted pulse, give a matrix $1000 \times n$. Each line of this matrix is a realization of the noisy signal.
$\rightsquigarrow$ Apply the matched filter to these 1000 realizations.


## Useful commands/tips:

The cross-correlation at 0 for all lines of a matrix : $c=1 / \mathrm{N} * \mathrm{X} * \mathrm{~s}$ '; (s line vector, X matrix, C column vector)
$\rightsquigarrow \quad$ Plot the histograms of the results under the two hypothesis H 0 and H 1 .

Q3 What is the influence of the threshold value on the detection probability (true positives) and on the false alarm probability (false positives)?

## Useful commands/tips:

To plot two histograms on the same graphic :
$[\mathrm{N} 1, \mathrm{X} 1]=$ hist $(c 1,20) ;[\mathrm{NO}, \mathrm{XO}]=$ hist $(\mathrm{co}, 20)$;
plot(X1, N1, XO, NO), legend('H1', 'HO')
$\rightsquigarrow \quad$ The ROC (Receiver Operating Characteristic) is a curve of the detection probability versus false alarm probability for all the threshold possible values. Plot the ROC curve from your Monte Carlo simulation.
$\rightsquigarrow$ Plot again the histograms and the ROC curve for simulation with SNR of 6 and 20 dB .
$\rightsquigarrow$ Plot now the ROC curves for SNR varying from 0 to 20 dB .

Q4 What is the limit of the ROC curve when SNR decreases? Why?



## 2 Object detection in a real-time video

The aim of this part is to detect an objet on a real time video from the webcam. First of all, you shall test your algorithm on good quality images.

### 2.1 Matched Filter on images

Here we wish to detect a character in a text. In this case we don't know its position. We must address the "detection-estimation" problem. The matched filter can be used for that purpose :

- a peak in the matched filter output indicate if the character is detected,
- the peak position gives an estimate of the character position.
$\rightsquigarrow$ Design and test such an algorithm on good quality images (parfait.bmp for instance). The character to detect will be selected on the image.



<font color="yellow">
First, we need to extract a perfect caracter from the perfect image in order to perform a correlation between the image and the selected caracter. Then, if the process is a success, we will perform the correlation between the same caracter and a noisy image to test the robustness of the matched filter.

In order to test both the detection and estimation performed by the match filter, we decide to draw a circle centered on each estimated position of detected correlation on the image.

</font>

In [518]:
from PIL import Image

# Function to load image as a 1D grayscale NumPy array
def load_image(image_path):
    """ChatGPT made function that loads into the workspace both bmp and png images and converts to grayscale.

    Args:
        image_path (string): Path of image.

    Returns:
        ndarray: Numpy 1D array (representation of grayscale loade image)
    """

    # Open the image file
    with Image.open(image_path) as img:
        # Convert the image to grayscale (L mode)
        img = img.convert('L')
        # Convert the grayscale image to a NumPy array
        img_array = np.array(img)
        # Flatten the 2D array into a 1D array
        #img_1d_array = img_array.flatten()

    return img_array



In [527]:
# Define ploting (2D) functions

def plot2d(z, titleFig, show, table_image, showImage):

    fig = go.Figure(data=[
        go.Heatmap(
            z=z, # , x=x, y=y
            opacity=0.4
        ) 
    ])

    fig.update_layout(
        title=titleFig,
        #autosize=False,
        #width=500,
        #height=500,
        margin=dict(l=65, r=50, b=65, t=90)
    )

    # Add the input variable table (image) to the plot as an annotation
    if showImage:
        fig.add_layout_image(
            dict(
                source=table_image,  # Use the loaded image
                xref="paper",  # Refer to the paper coordinates (0 to 1)
                yref="paper",
                x=1.1,         # Positioning to the right of the plot (1.1 means slightly to the right)
                y=1,           # Positioning at the top of the plot
                sizex=0.3,     # Adjust the size of the image (horizontal)
                sizey=0.3,     # Adjust the size of the image (vertical)
                xanchor="left",  # Anchor the image's left side
                yanchor="top"   # Anchor the image's top side
            )
        )
    else:
        pass

    if show:
        fig.show()
    else:
        pass

    return fig

In [528]:
import io
import base64

def run2():

    # Import necessary image files
    sRef = load_image("Donnees_pour_les_TP_20240920/DonneesEtImages/s.png")
    uRef = load_image("Donnees_pour_les_TP_20240920/DonneesEtImages/u.png")

    image = load_image("Donnees_pour_les_TP_20240920/DonneesEtImages/parfait.bmp")

    table_image = Image.open("Donnees_pour_les_TP_20240920/DonneesEtImages/parfait.bmp")
    # Convert the image to base64
    buffered = io.BytesIO()
    table_image.save(buffered, format="PNG")  # Convert BMP to PNG in-memory
    encoded_image = base64.b64encode(buffered.getvalue()).decode('utf-8')

    sDE = sp.signal.correlate2d(sRef, image, mode="same", boundary="fill", fillvalue=0)
    uDE = sp.signal.correlate2d(uRef, image, mode="same", boundary="fill", fillvalue=0)

    sDEFig = plot2d(sDE, "Correlation function across the image for 's' caracter", True, encoded_image, True)
    uDEFig = plot2d(uDE, "Correlation function across the image for 'u' caracter", True, encoded_image, True)

if __name__=="__main__":
    run2()


## Useful commands/tips:

- imread read image from graphics file, example:
a=imread('parfait.bmp').
- double convert to double precision, example ad=double(a);
- $a=m e a n(m e a n(a))-a$; by this command a white background image $a$ is transformed in a zero mean, black background one. Use this command before computing the matched filter.
- imcrop crops an image to a specified rectangle.
- In order to seek peak positions and display a red 'x-mark' on each :
$[\mathrm{I}, \mathrm{J}]=\mathrm{find}(\operatorname{cs} 2>0)$;
figure, imshow(text), hold on, plot(J,I,'xr');


### 2.2 Real-time Video

$\rightsquigarrow$ Set the webcam resolution to 640x480pixels, using the Logitech software.
$\rightsquigarrow$ Execute the program WebcamImageCapture and take a look at the program file by using the command edit WebcamImageCapture.m.

Q5 What is the name, the size and the format of the acquired image?
$\rightsquigarrow \quad$ By copying the source code of the program, design a program which can detect an track an object (of your choice) in the webcam image.



## Useful commands/tips:

- rgb2gray convert a color image on a (8 bits) gray-level image.
- WebcamImageCapture.m is a script which acquire and display images from the webcam using the Image Acquisition Toolbox.